In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split   # 数据集划分
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV  # 带交叉验证的网格搜索
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix
from vocabulary import Vocabulary
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from time import time
import datetime
import pickle
import warnings

warnings.filterwarnings("ignore")

## 1、特征为residue_vocabulary，其序列残基对应词汇表所做的字符编码

In [2]:
# 将序列残基对应词典转换成特征
data = pd.read_excel('/home/dldx/R-H/code/classification/ML/data/mafft_MSA_R_H.xlsx')

vocabulary = Vocabulary.get_vocabulary_from_sequences(data.Sequence.values)  #将蛋白质序列构造成词典

In [3]:
input_seq_tensor = []
input_lable = []
for i in range(179):
    # 将序列残基进行字符编码
    seq_tensor = vocabulary.seq_to_tensor(data.Sequence[i])
    input_seq_tensor.append(np.array(seq_tensor))
    
input_seq_tensor = np.array(input_seq_tensor)
input_lable = np.array(data.Tier)

In [71]:
# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["linear","poly","rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}
# Kernel = ["linear","poly","rbf","sigmoid"]
for r in range(31):
    # 划分训练集和测试集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = r)
#     for kernel in Kernel:
    time0 = time()
    clf= SVC(degree = 1
             , cache_size=5000
             , random_state = 30
             )
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    clf = clf.fit(Xtrain,Ytrain)

    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r)  + "，定义SVM模型 random_state：" + str(30) + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.7035243599527023
  sens : 73.33333333333333
  spec : 90.47619047619048
数据集划分 random_state：1，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.44272445820433437
  sens : 70.58823529411765
  spec : 73.68421052631578
数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.7559289460184544
  sens : 40.0
  spec : 100.0
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.6324555320336759
  sens : 87.5
  spec : 80.0
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.38550399347880293
  sens : 90.9090909090909
  spec : 64.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7527726527090811
  sens : 31.57894736842105
  spec : 100.0
数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate

## 2、特征为residue_disorder，其序列残基对应disorder所做的字符编码

In [72]:
data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')
input_lable1 = np.array(data1.Tier)
input_feature = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/all_seq_disorder_R_H.pkl','rb'))

In [73]:
# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["linear","poly","rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}

for r in range(31):
    # 划分训练集和测试集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = r)
    clf= SVC(degree = 1
             , cache_size=5000
             , random_state = 30
             )
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    clf = clf.fit(Xtrain,Ytrain)

    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r)  + "，定义SVM模型 random_state：" + str(30) + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.22576567886036575
  sens : 26.666666666666668
  spec : 85.71428571428571
数据集划分 random_state：1，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.44272445820433437
  sens : 70.58823529411765
  spec : 73.68421052631578
数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.5170907197968263
  sens : 65.0
  spec : 81.25
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.31622776601683794
  sens : 68.75
  spec : 65.0
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.4626219273003039
  sens : 81.81818181818183
  spec : 76.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.5071199432577871
  sens : 47.368421052631575
  spec : 88.23529411764706
数据集划分 random_state：6，定义SVM模型 random_st

## 3、特征为21个无规则卷曲的关键区域长度，IDR1, IDR2, IDR3, IDR4, IDR5, IDR6, IDR7, IDR8,IDR9,IDR10,IDR11,IDR12,IDR13,IDR14,IDR15,IDR16,IDR17,IDR18,IDR19,IDR20,IDR21。

In [4]:
data2_R = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_R_1.pkl','rb'))
data2_H = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_H_3.pkl','rb'))

In [5]:
data2_R[65][15]

3

In [6]:
all_21_IDR = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in data2_R]
    H_IDR = [i[position] for i in data2_H]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR添加到一个数组
    all_21_IDR.append(one_IDR)

In [7]:
# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}
for position in range(21):
    for r in range(0,31):
        # 划分训练集和测试集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[position],input_lable,train_size=0.8,random_state = r)
        clf= SVC(degree = 1
                 , cache_size=5000
                 , random_state = 30
                 )
        # 将模型加入超参数网格搜索和交叉验证     
        clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
        clf = clf.fit(Xtrain,Ytrain)

        Y_pred = clf.predict(Xtest)
        cm = confusion_matrix(Ytest,Y_pred)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        print("数据集划分 random_state：" + str(r)  + "，定义SVM模型 random_state：" + str(30) + "时，")
        print("Evaluate：")
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")  
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0

数据集划分 random_state：20，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：21，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.09072184232530289
  sens : 22.22222222222222
  spec : 72.22222222222221
数据集划分 random_state：22，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：23，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.14704997960062352
  sens : 14.285714285714285
  spec : 77.27272727272727
数据集划分 random_state：24，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：25，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.26038545756294446
  sens : 7.142857142857142
  spec : 81.81818181818183
数据集划分 random_state：26，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc 

数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.3491486243775878
  sens : 55.00000000000001
  spec : 75.0
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5
  mcc : -0.0125
  sens : 43.75
  spec : 55.00000000000001
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4526019054848144
  sens : 18.181818181818183
  spec : 96.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.18961876139204215
  sens : 36.84210526315789
  spec : 76.47058823529412
数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : -0.16883116883116883
  sens : 28.57142857142857
  spec : 54.54545454545454
数据集划分 random_state：7，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.09209405303147204
  sens : 50.0
  spec : 64.28571428571429
数据集划分 random_state：8，定义SVM模型 random_state：30时，
Evalu

数据集划分 random_state：14，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3779644730092272
  sens : 40.0
  spec : 85.71428571428571
数据集划分 random_state：15，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.35195307605124787
  sens : 25.0
  spec : 91.66666666666666
数据集划分 random_state：16，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.75
  mcc : 0.6471789394828489
  sens : 38.46153846153847
  spec : 95.65217391304348
数据集划分 random_state：17，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.29649972666444047
  sens : 40.0
  spec : 84.61538461538461
数据集划分 random_state：18，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : 0.32641044530137714
  sens : 18.181818181818183
  spec : 92.85714285714286
数据集划分 random_state：19，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.4187628431947846
  sens : 30.76923076923077
  spec : 91.30434782608695
数据集划分 random_state：20，定

数据集划分 random_state：26，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.8401680504168059
  sens : 16.666666666666664
  spec : 100.0
数据集划分 random_state：27，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.3241018617760822
  sens : 12.5
  spec : 96.42857142857143
数据集划分 random_state：28，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.8224783208299743
  sens : 15.384615384615385
  spec : 100.0
数据集划分 random_state：29，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.37113480951260275
  sens : 14.285714285714285
  spec : 95.45454545454545
数据集划分 random_state：30，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
以上为IDR__6时，evaluation
***********************************************************************
数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc :

...

数据集划分 random_state：23，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：24，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：25，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.504524979109513
  sens : 0.0
  spec : 95.45454545454545
数据集划分 random_state：26，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.21004201260420147
  sens : 8.333333333333332
  spec : 95.83333333333334
数据集划分 random_state：27，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：28，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30，定义SVM模型 r

数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.2807449625374386
  sens : 0.0
  spec : 75.0
数据集划分 random_state：8，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：9，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：10，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：12，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.61

数据集划分 random_state：22，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.2984810028978546
  sens : 63.63636363636363
  spec : 72.0
数据集划分 random_state：23，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.4103049699311091
  sens : 42.857142857142854
  spec : 86.36363636363636
数据集划分 random_state：24，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.26016534852418005
  sens : 28.57142857142857
  spec : 86.36363636363636
数据集划分 random_state：25，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.26016534852418005
  sens : 28.57142857142857
  spec : 86.36363636363636
数据集划分 random_state：26，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.19245008972987526
  sens : 66.66666666666666
  spec : 58.333333333333336
数据集划分 random_state：27，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.19241827716833385
  sens : 50.0
  spec : 75.0
数据集划分 r

数据集划分 random_state：7，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4166666666666667
  mcc : -0.31524416249564025
  sens : 0.0
  spec : 53.57142857142857
数据集划分 random_state：8，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：10，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.077138921583987
  sens : 33.33333333333333
  spec : 72.22222222222221
数据集划分 random_state：12，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3424747597107866
  sens : 35.714285714285715
  spec : 86.36363

数据集划分 random_state：23，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142857142
  spec : 100.0
数据集划分 random_state：24，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.015316791621349673
  sens : 14.285714285714285
  spec : 86.36363636363636
数据集划分 random_state：25，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：26，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：27，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.8944271909999159
  sens : 12.5
  spec : 100.0
数据集划分 random_state：28，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918
  sens : 7.142857142

In [7]:
# 重新跑关于IDR16由数据更改的情况

# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}
for r in range(0,31):
    # 划分训练集和测试集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[15],input_lable,train_size=0.8,random_state = r)
    clf= SVC(degree = 1
             , cache_size=5000
             , random_state = 30
             )
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    clf = clf.fit(Xtrain,Ytrain)

    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r)  + "，定义SVM模型 random_state：" + str(30) + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")  
print("以上为IDR__" + str(15 + 1) + "时，evaluation")

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : -0.3070961343199971
  sens : 15.0
  spec : 68.75
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5833333333333334
  m

In [8]:
# 将21个区域的无规则卷曲长度一起作为特征
all_R_H_IDR =[]
all_R_H_IDR.extend(data2_R)
all_R_H_IDR.extend(data2_H)

# 设置超参数
param_dict = {"C":[10 ** i for i in range(-5, 6)], "kernel": ["rbf","sigmoid"], "gamma": np.logspace(-10, 1, 50),
              "decision_function_shape":["ovo", "ovr"]}
for r in range(31):
    # 划分训练集和测试集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR,input_lable,train_size=0.8,random_state = r)
    clf= SVC(degree = 1
             , cache_size=5000
             , random_state = 30
             )
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    clf = clf.fit(Xtrain,Ytrain)

    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    print("数据集划分 random_state：" + str(r)  + "，定义SVM模型 random_state：" + str(30) + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")  

数据集划分 random_state：0，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.34518102637329706
  sens : 46.666666666666664
  spec : 80.95238095238095
数据集划分 random_state：1，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.3278050340535929
  sens : 47.05882352941176
  spec : 78.94736842105263
数据集划分 random_state：2，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.7071067811865475
  sens : 20.0
  spec : 100.0
数据集划分 random_state：3，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.27325202042558927
  sens : 56.25
  spec : 70.0
数据集划分 random_state：4，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.35
  sens : 90.9090909090909
  spec : 60.0
数据集划分 random_state：5，定义SVM模型 random_state：30时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.4661384642714042
  sens : 42.10526315789473
  spec : 88.23529411764706
数据集划分 random_state：6，定义SVM模型 random_state：30时，
Evaluate